In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mojimoji
import re
import jaconv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
from sklearn import preprocessing
import MeCab
def parse_text(text, debug=False):
    text = re.sub(r'［[^］]+］', ' ', text)  
#     text = re.sub(r'（[^）]+）', ' ', text)  
    text = re.sub(r'○', ' ', text)    
    text = re.sub(r'×', '', text)
    text = re.sub(r'※', '', text)    
#     text = re.sub(r'｜', ' ', text)
    text = re.sub(r'[\s、]', '' , text)
#     text = re.sub(r'一', ' ', text)
#     text = re.sub(r'…', ' ', text)  
#     text = re.sub(r'―', ' ', text)
    text = re.sub(r'[0-9]', '0', text)
    return text

def tokenize1(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)
y = df['author'].values
mecab = MeCab.Tagger('-Owakati')
count = TfidfVectorizer(analyzer=tokenize1)
bags = count.fit_transform(df.body.values)

features = count.get_feature_names()

bodyvec = pd.DataFrame(bags.toarray(), columns=features)
# wordbody=bodyvec.sum().sort_values(ascending=False)
# bodyvec1 = bodyvec[list(wordbody[wordbody > 80].index)]
# xsum = bodyvec1.sum(axis=1)

# xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec)], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)


In [2]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 0) # 80%のデータを学習データに、20%を検証データにする
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(train_X,train_y)
pred = np.where(mlpr.predict(test_X)>0.38,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[919   0]
 [ 14  61]]
accuracy =  0.9859154929577465
precision =  1.0
recall =  0.8133333333333334
f1 score =  0.8970588235294117


In [9]:
pred = np.where(mlpr.predict(test_X)>0.16,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[919   0]
 [  2  73]]
accuracy =  0.9979879275653923
precision =  1.0
recall =  0.9733333333333334
f1 score =  0.9864864864864865


In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=42, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [25]:
pred = np.where(mlpr.predict(test)>0.29,1,0)
pred=mlpr.predict(test)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [11]:
pred=mlpr.predict(test)

In [15]:
target=pd.concat([y,pd.Series(pred)])

AttributeError: 'Series' object has no attribute 'columns'

In [18]:
target=pd.concat([y,pd.Series(pred)])
data=pd.concat([pd.concat([pd.DataFrame(X),pd.DataFrame(test)]),pd.concat([y,pd.Series(pred)])],axis=1)
data.columns=features+["target"]
data

,,!,!!,!!”,"!""",!.....,!――,!――――,!”,!”「…………」,...,｜,｜│,｝,～,～～。,｣,￡,￡―,￥,target
0,0.143511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.090834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.194287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.029252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.073963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,0.027669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.047519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002840
1416,0.007975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.133112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003049
1417,0.289270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027068
1418,0.008398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.086165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011124


In [19]:
data[(data.target<0.7)&(data.target>0.29)].target
#19 1
#88 1
#96 1 0.31
#135 1
#138 1
#141 1 0.4
#142 1 0.5
#165 1 0.45
#167 1 0.66
#183 1 0.47
#186 1 0.66
#218 1 0.39
#258 1 0.39
#287 1 0.44
#335 1 0.58
#341 1 0.69
#343 1 0.35
#380 1 

19      0.536267
88      0.602552
96      0.318674
135     0.396326
138     0.537502
141     0.402134
142     0.548170
165     0.451639
167     0.667873
183     0.475976
185     0.580187
186     0.663790
218     0.391250
258     0.403657
287     0.443843
335     0.583314
341     0.698047
343     0.358028
380     0.594235
384     0.306984
435     0.509139
442     0.426857
443     0.617322
452     0.329839
469     0.620779
485     0.588348
497     0.606865
521     0.620151
526     0.558754
576     0.649340
599     0.538842
610     0.521236
621     0.517787
659     0.560696
693     0.509380
699     0.452814
715     0.569356
721     0.449883
784     0.556094
788     0.551037
794     0.392184
823     0.687583
856     0.631055
922     0.589146
965     0.526702
1003    0.593013
1038    0.532951
1146    0.565096
1194    0.403040
1268    0.673910
1322    0.690617
1343    0.580035
1361    0.469842
1363    0.505366
1406    0.551303
Name: target, dtype: float64

In [20]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
19,57,「貴君《あなた》の作品の中《うち》で愛着を持つてゐらつしやるものか好きなものはありませんか」...
88,282,上何小二《かしょうじ》は軍刀を抛《ほう》り出すと夢中で馬の頸《くび》にしがみついた。確かに頸...
96,328,久米は官能の鋭敏な田舎者です。書くものばかりじゃありません。実生活上の趣味でも田舎者らしい所...
135,474,立てきった障子《しょうじ》にはうららかな日の光がさして嵯峨《さが》たる老木の梅の影が何間《な...
138,488,佐佐木君は剛才人小島君は柔才人兎に角どちらも才人です。僕はいつか佐佐木君と歩いていたら佐佐木...
141,495,野呂松人形《のろまにんぎょう》を使うから見に来ないかと云う招待が突然来た。招待してくれたのは...
142,497,一ある春の午《ひる》過ぎです。白《しろ》と云う犬は土を嗅《か》ぎ嗅ぎ静かな往来を歩いていまし...
165,569,皆さん。私《わたし》は今大阪にいますですから大阪の話をしましょう。昔大阪の町へ奉公《ほうこう...
167,571,天草《あまくさ》の原《はら》の城の内曲輪《うちくるわ》。立ち昇る火焔。飛びちがふ矢玉。伏し重...
183,636,昔支那《シナ》の或《ある》田舎に書生《しょせい》が一人住んでいました。何しろ支那のことですか...


In [49]:
data[(data.target<0.7)&(data.target>0.29)].target
#291 0 0.31
#412 1 0.61
#547 1 0.39 ✖︎
#625 1 0.34 ✖︎
#682 0 0.54 ✖︎
#702 1 0.56 
#774 0 0.37 
#815 1 0.35 ✖︎
#825 0 0.32 
#955 1 0.59
#962 0 0.29
#1029 1 0.42 ✖︎
#1050 0 0.29
#1116 1 0.60
#1140 1 0.31 ✖︎
#1194 1 0.65
#1200 0 0.42 
#1281 0 0.44 

291     0.311938
412     0.615388
547     0.394943
625     0.348613
682     0.540990
702     0.567220
774     0.376710
815     0.357455
825     0.321671
955     0.598116
962     0.290931
1029    0.428522
1050    0.290529
1116    0.609978
1140    0.315641
1194    0.653080
1200    0.428232
1281    0.442822
Name: target, dtype: float64

In [14]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
291,998,一私が生れて始めて蓄音器と云うものを見聞いたのはもう十四五年前父が英国から土産に買って来たも...
412,1379,昔々バグダツドのマホメツト教のお寺の前に一人の乞食が寝て居りました。丁度その時説教がすんだの...
547,1862,或る一つの作品を書かうと思つてそれが色々の径路を辿《たど》つてから出来上がる場合と直ぐ初めの...
625,2137,菊池は生き方が何時も徹底している。中途半端のところにこだわっていない。彼自身の正しいと思うと...
682,2321,十年前は阿佐ヶ谷に住んで居りやはり目下と同様吶々と小説ばかり書いて居りました。もう十年も経つ...
702,2385,すべて背景を用いない。宦官《かんがん》が二人話しながら出て来る。――今月も生み月になっている...
774,2649,ナポレオンの遺書――セント・ヘレナの島で臨終より三週間ほど前に彼が自ら口述し浄書したもので現...
815,2791,僕はこれからも今月のと同じような材料を使って創作するつもりである。あれを単なる歴史小説の仲間...
825,2828,一崇拝する偉人一現存の人の裡世界第一の偉人と思惟する人一右理由一プラトーシェイキスピアゲーテ...
955,3204,一ロマンスの中の女性は善悪共皆好み候。二あゝ云ふ女性は到底この世の中にゐないからに候。


In [71]:
testdf.body[1082]

'長二《ちょうじ》は貧乏《びんぼう》の家《いえ》に生《う》まれておもちゃも持《も》たずに死《し》んでしまった。美《うつく》しいガラス張《ば》りの店頭《みせさき》に西洋《せいよう》のぜいたくな小間物《こまもの》や赤《あか》紫《むらさき》に塗《ぬ》ったゴムまりやぴかぴかと顔《かお》の映《うつ》る銀笛《ぎんてき》やらっぱやなんでも子供《こども》の好《す》きそうなものが並《なら》べてあるのを見《み》ると店《みせ》のガラス戸《ど》を砕《くだ》いてそれらのものをめちゃめちゃにたたき壊《こわ》してやりたくなる。隣《となり》に住《す》んでいたあの貧《まず》しかった哀《あわ》れな長二《ちょうじ》のことを思《おも》い出《だ》したときに。'